<a href="https://colab.research.google.com/github/francisco140472/tamanduatei/blob/main/JOTFORM_VU_TAMANDUATI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install schedule
!pip install paramiko

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 856.7/856.7 kB 45.5 MB/s eta 0:00:00


In [ ]:
import requests
import pandas as pd
import folium
from io import BytesIO
import paramiko
import time
from datetime import datetime
import matplotlib.pyplot as plt
import base64

# URL para a API do Jotform para pegar o arquivo Excel
url = "https://www.jotform.com/excel/242924996615066"

# Fazendo o download do arquivo Excel
response = requests.get(url)

# Verificando se o download foi bem-sucedido
if response.status_code == 200:
    # Lê o conteúdo do Excel diretamente no pandas
    df = pd.read_excel(BytesIO(response.content))
else:
    print(f"Error: {response.status_code}")

# Remove registros onde NÚCLEO_DE_ATUAÇÃO é NaN
df = df.dropna(subset=['NÚCLEO_DE_ATUAÇÃO'])

# Contagem de registros não nulos em cada coluna para verificar o total de registros em cada uma
total_ocorrencias = df['OCORRÊNCIA DE CAMPO CADASTRO'].notna().sum()
soma_caixa_uma = df['OCORRÊNCIA DE CAMPO CAIXA UMA'].notna().sum()
total_quantidade_instalacao = df['OCORRÊNCIA DE CAMPO LIGAÇÃO'].notna().sum()

# Agrupamento por NÚCLEO_DE_ATUAÇÃO
localizacoes_por_bairro = {}
for bairro in df['NÚCLEO_DE_ATUAÇÃO'].unique():
    dados_bairro = df[df['NÚCLEO_DE_ATUAÇÃO'] == bairro]
    localizacoes = []
    for index, row in dados_bairro.iterrows():
        if pd.notna(row['Geolocation']) and ',' in row['Geolocation']:
            try:
                lat, lng = row['Geolocation'].split(',')
                lat = float(lat.strip())
                lng = float(lng.strip())
                foto_url = row.get('FOTO_DO_IMOVEL', 'https://via.placeholder.com/150')
                if not foto_url:
                    foto_url = 'https://via.placeholder.com/150'  # Link para a imagem padrão
                localizacoes.append((lat, lng, row['BAIRRO'],
                                     row.get('FUNCIONÁRIO CADASTRO', 'Não informado'),
                                     row.get('OCORRÊNCIA DE CAMPO CADASTRO', 'Não informada'),
                                     row.get('IDENTIFICAÇÃO ÚNICA', 'Não informada'),
                                     foto_url
                ))
            except ValueError:
                print(f"Erro ao converter coordenadas: {row['Geolocation']}")
        else:
            print(f"Geolocalização inválida ou ausente para o registro: {index}")

    localizacoes_por_bairro[bairro] = localizacoes

# Define as cores e ícones para cada bairro
cores_icones = ['red', 'blue', 'green', 'purple', 'orange', 'darkred', 'lightred', 'beige', 'darkblue', 'darkgreen']
icons = ['info-sign', 'cloud', 'leaf', 'star', 'ok', 'bookmark', 'cutlery', 'coffee', 'road', 'home']
cores_bairro = {bairro: (cores_icones[i % len(cores_icones)], icons[i % len(icons)]) for i, bairro in enumerate(localizacoes_por_bairro)}

# Cria um mapa centrado em uma localização genérica
m = folium.Map(location=[-23.55, -46.63], zoom_start=12)

# Adiciona título no mapa
title_html = '''
    <h3 align="center" style="font-size:20px"><b>VU - TAMANDUATEI - ENORSIG</b></h3>
'''
m.get_root().html.add_child(folium.Element(title_html))

# Adiciona a tabela de resumo ao lado esquerdo
tabela_resumo_html = '''
<div style="position: fixed;
            top: 80px; left: 50px; width: 300px;
            background-color: white;
            padding: 10px;
            border: 2px solid grey;
            z-index: 9999;">
    <h4>Resumo de Ocorrências</h4>
    <table style="width: 100%; border-collapse: collapse; text-align: center;">
        <tr>
            <th style="border: 1px solid black; padding: 5px; text-align: center;">OCORRÊNCIAS</th>
            <th style="border: 1px solid black; padding: 5px; text-align: center;">CAIXA UMA</th>
            <th style="border: 1px solid black; padding: 5px; text-align: center;">LIGAÇÕES</th>
        </tr>
        <tr>
            <td style="border: 1px solid black; padding: 5px; text-align: center;">{total_ocorrencias}</td>
            <td style="border: 1px solid black; padding: 5px; text-align: center;">{soma_caixa_uma}</td>
            <td style="border: 1px solid black; padding: 5px; text-align: center;">{total_quantidade_instalacao}</td>
        </tr>
    </table>
</div>
'''

# Adiciona a tabela de resumo ao mapa com os valores formatados
m.get_root().html.add_child(folium.Element(tabela_resumo_html.format(
    total_ocorrencias=total_ocorrencias,
    soma_caixa_uma=soma_caixa_uma,
    total_quantidade_instalacao=total_quantidade_instalacao
)))

# Adicionar contagem por tipo de "OCORRÊNCIA DE CAMPO CADASTRO"
total_por_tipo = df['OCORRÊNCIA DE CAMPO CADASTRO'].value_counts()

# Adicionar contagem de cadastro por dia, mantendo apenas a última data
df['DATA CADASTRO'] = pd.to_datetime(df['DATA CADASTRO'], errors='coerce')
total_por_dia = df.groupby(df['DATA CADASTRO'].dt.date).size()
ultima_data = total_por_dia.index[-1] if not total_por_dia.empty else None
total_ultima_data = total_por_dia[ultima_data] if ultima_data else 0

# Tabela HTML com total por tipo e apenas a última data, abaixo da primeira tabela
tabela_tipo_ultima_data_html = '''
<div style="position: fixed;
            top: 320px; left: 50px; width: 300px;
            background-color: white;
            padding: 10px;
            border: 2px solid grey;
            z-index: 9999;">
    <h4>Resumo por Tipo</h4>
    <table style="width: 100%; border-collapse: collapse; text-align: center;">
        <tr>
            <th style="border: 1px solid black; padding: 5px; text-align: center;">Tipo de Ocorrência</th>
            <th style="border: 1px solid black; padding: 5px; text-align: center;">Total</th>
        </tr>''' + ''.join([f'''
        <tr>
            <td style="border: 1px solid black; padding: 5px;">{tipo}</td>
            <td style="border: 1px solid black; padding: 5px;">{total}</td>
        </tr>''' for tipo, total in total_por_tipo.items()]) + '''
    </table>
    <h4>Última Data</h4>
    <table style="width: 100%; border-collapse: collapse; text-align: center;">
        <tr>
            <th style="border: 1px solid black; padding: 5px; text-align: center;">Data</th>
            <th style="border: 1px solid black; padding: 5px; text-align: center;">Total</th>
        </tr>
        <tr>
            <td style="border: 1px solid black; padding: 5px;">{ultima_data}</td>
            <td style="border: 1px solid black; padding: 5px;">{total_ultima_data}</td>
        </tr>
    </table>
</div>
'''

# Adiciona a tabela de tipo e última data ao mapa
m.get_root().html.add_child(folium.Element(tabela_tipo_ultima_data_html.format(
    ultima_data=ultima_data.strftime('%Y-%m-%d') if ultima_data else 'N/A',
    total_ultima_data=total_ultima_data
)))

# Adicionar marcadores para cada núcleo e camada
for bairro, localizacoes in localizacoes_por_bairro.items():
    cor, icone = cores_bairro[bairro]
    feature_group = folium.FeatureGroup(name=bairro)

    for lat, lng, bairro, funcionario, cadastro, identificacao, foto_url in localizacoes:
        # Adiciona o marcador para cada localização com ícone e cor personalizada
        popup_html = f"""
            <div>
                <h4>{bairro}</h4>
                <p><b>Funcionário:</b> {funcionario}</p>
                <p><b>Cadastro:</b> {cadastro}</p>
                <p><b>Identificação Única:</b> {identificacao}</p>
                <img src="{foto_url}" alt="Foto do Imóvel" width="100%">
            </div>
        """
        folium.Marker(
            location=[lat, lng],
            popup=folium.Popup(popup_html, max_width=300),
            icon=folium.Icon(color=cor, icon=icone)
        ).add_to(feature_group)

    feature_group.add_to(m)

# Adiciona controle de camadas ao mapa
folium.LayerControl().add_to(m)

# Exibir mapa
#m
 # Assumindo que você já definiu o objeto `m` antes
m.save("VU_TAMANDUATEI_ENORSIG.html")
print("HTML gerado com sucesso!")
#########################################################################################
# Função para enviar o HTML para o servidor via SFTP
def enviar_para_servidor():
    try:
        # Configurações de conexão SSH
        ssh = paramiko.SSHClient()
        ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
        ssh.connect('servidor_oul.com', username='usuario', password='senha')  # Substitua pelas credenciais corretas

        # Enviar o arquivo
        sftp = ssh.open_sftp()
        sftp.put('VU_TAMANDUATEI_ENORSIG.html', '/caminho/servidor/VU_TAMANDUATEI_ENORSIG.html')
        sftp.close()
        ssh.close()
        print(f"{datetime.now()}: Arquivo enviado ao servidor com sucesso!")

    except Exception as e:
        print("Erro ao enviar para o servidor:", e)

# Loop de atualização a cada 15 minutos
print("Iniciando atualização automática a cada 15 minutos...")
while True:
    try:
        gerar_html()              # Gera o HTML
        enviar_para_servidor()    # Envia o arquivo gerado para o servidor
    except Exception as e:
        print("Erro durante a atualização:", e)

    # Aguarda 15 minutos antes de repetir
    time.sleep(15 * 60)


Geolocalização inválida ou ausente para o registro: 1052
Geolocalização inválida ou ausente para o registro: 1295
Geolocalização inválida ou ausente para o registro: 2054
Geolocalização inválida ou ausente para o registro: 2092
HTML gerado com sucesso!
Iniciando atualização automática a cada 15 minutos...
Erro durante a atualização: [Errno 2] No such file or directory: '/content/<seu-repositorio>/VU_TAMANDUATEI_ENORSIG.html'
